Импортируем нужные библиотеки.

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

Распакуйте архив с данными в папку,где находится этот jupyter notebook (baseline.ipynb). У вас будет папка data,  содержащая необходимые файлы. 

Ваша задача предсказать вероятность совершения покупки Клиентом в определенных 8 категориях в следующие 7 дней, чтобы под них Банк смог направить релевантный контент (подборки). Для подготовки предсказания у вас есть данные по транзакциям 50 000 Клиентов в течение года, из которых 25 000 – в обучающей выборке и 25 000 – в тестовой. Ваша задача для тестовой выборки рассчитать для каждой из целевой категории вероятность совершения покупки в ней на следующей неделе.

Данные по транзакциям находятся в файле **transactions_train.csv**. Информация о покупках по категориям для исследуемой недели находится в файле **train_target.csv**.

Считаем данные по транзакциям и правильные ответы.

In [ ]:
transactions_train=pd.read_csv('transactions_train.csv')

In [ ]:
train_target=pd.read_csv('train_target.csv')

Посмотрим на данные.

In [ ]:
transactions_train.head(5)

* client_dk - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* amount - сумма транзакции
* small_group - категория покупки

In [ ]:
train_target.head(5)

* client_dk - уникальный идентификатор клиента, соответствует полю client_dk из транзакций
* числовые названия колонок - это 8 категорий продуктов. Их названия (числа) соответствуют значениям в колонке **small_group** из данных по транзакциям. Значения в этих колонках бинарные,т.е. 0 - в исследуемую неделю не было совершено покупки в данной категории, 1 - покупка была совершена. Например, клиент с номером 34110 (первая строчка) купил товар из категории 45, а по остальным категориям покупок не совершал.

Далее представлен простой вариант решения задачи. Вы можете решать соревнование используя совершенно другой подход.

Посчитаем по каждому клиенту самые простые аггрегационные признаки.

In [ ]:
agg_features=transactions_train.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()

In [ ]:
agg_features.head(5)

Посчитаем для каждого клиента количество транзакций по каждой категории.

In [ ]:
counter_df_train=transactions_train.groupby(['client_dk','small_group'])['amount'].count()

In [ ]:
cat_counts_train=counter_df_train.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')

In [ ]:
cat_counts_train=cat_counts_train.fillna(0)

In [ ]:
cat_counts_train.columns=['small_group_'+str(i) for i in cat_counts_train.columns]

In [ ]:
cat_counts_train.head()

Далее соединим все файлы в один датафрейм с таргетом.

In [ ]:
train=pd.merge(train_target,agg_features,on='client_dk')

In [ ]:
train=pd.merge(train,cat_counts_train.reset_index(),on='client_dk')

In [ ]:
train.head()

Теперь подгрузим тестовые данные для того, чтобы сделать предсказание. Проделаем с ними те же самые манипуляции, как и с обучающими данными.

In [ ]:
transactions_test=pd.read_csv('transactions_test.csv')

Также загрузим id тестовых клиентов, по которым нужно сделать предсказание.

In [ ]:
test_id=pd.read_csv('test.csv')

In [ ]:
agg_features_test=transactions_test.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()

In [ ]:
counter_df_test=transactions_test.groupby(['client_dk','small_group'])['amount'].count()

In [ ]:
cat_counts_test=counter_df_test.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')

In [ ]:
cat_counts_test=cat_counts_test.fillna(0)

In [ ]:
cat_counts_test.columns=['small_group_'+str(i) for i in cat_counts_test.columns]

In [ ]:
cat_counts_test.head()

In [ ]:
test=pd.merge(test_id[['client_dk']],agg_features_test,on='client_dk')

In [ ]:
test=pd.merge(test,cat_counts_test.reset_index(),on='client_dk')

In [ ]:
common_features=list(set(train.columns).intersection(set(test.columns)))

In [ ]:
X_train=train[common_features]
X_test=test[common_features]

В этом бэйзлайне мы будем использовать простой подход - предсказывать покупки в каждой категории независимо. То есть в цикле модель обучается на отдельную категорию как на зависимую переменную, и пытается предсказать наличие покупки в этой определенной категории для теста. В итоге у нас получается 8 задач бинарной классификации.

**Важно**: Такой подход не претендует на звание лучшего, вы вольны придумать свой алгоритм решения, который, вполне вероятно, окажется лучше.

In [ ]:
#В словарь будем записывать предсказания модели
results_tree = {}
#Цикл со второго элемента, потому что первой колонкой идет идентификатор клиента 
for q in train_target.columns[1:]:
    print('train product '+str(q))
    curr_target_train = train_target.loc[:,q]
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train.fillna(0).values,curr_target_train.values)
    #Сделаем предсказание
    pred = model.predict_proba(X_test.fillna(0).values)[:,1]
    results_tree[q] = pred

Такое решение дает на публичном лидерборде качество 0.6023

### Подготовим файл для отправки в систему

In [ ]:
submission = pd.DataFrame(data=np.zeros((25000,8)),columns=train_target.columns[1:],index=test_id['client_dk'].values)

In [ ]:
for q in results_tree:
    submission[q] = results_tree[q]

In [ ]:
submission.index.name = 'client_dk'
submission.columns = ['cat_27','cat_32','cat_41','cat_45','cat_67','cat_73','cat_81','cat_88']

Сохраняем прогноз на диск в папку submissions. Имя прогноза соответсвует дате и времени его создания, закодированными с помощью timestamp.

In [ ]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv(submission_path, index=True)

Теперь все готово! Можно отправлять решение.